In [1]:
import numpy as np
import pandas as pd


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv("Crop_recommendation.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Crop_recommendation.csv'

In [ ]:

land_area = {
    'rice': '0.5–2 ha',
    'maize': '0.5–2 ha',
    'chickpeas': '0.25–1 ha',
    'kidneybeans': '0.25–1 ha',
    'pigeonpeas': '0.25–1 ha',
    'mothbeans': '0.25–1 ha',
    'mungbean': '0.25–1 ha',
    'blackgram': '0.25–1 ha',
    'lentil': '0.25–1 ha',
    'pomegranate': '1–5 ha',
    'banana': '1–5 ha',
    'mango': '2–10 ha',
    'grapes': '1–5 ha',
    'watermelon': '0.5–2 ha',
    'muskmelon': '0.5–2 ha',
    'apple': '2–10 ha',
    'orange': '2–10 ha',
    'papaya': '1–5 ha',
    'coconut': '2–10 ha',
    'cotton': '2–10 ha',
    'jute': '2–10 ha',
    'coffee': '2–10 ha'
}
water_need = {
    'rice': 'tinggi',
    'banana': 'tinggi',
    'watermelon': 'tinggi',
    'muskmelon': 'tinggi',
    'grapes': 'tinggi',
    'maize': 'sedang',
    'cotton': 'sedang',
    'coffee': 'sedang',
    'papaya': 'sedang',
    'orange': 'sedang',
    'mango': 'sedang',
    'apple': 'sedang',
    'chickpeas': 'rendah',
    'lentil': 'rendah',
    'mungbean': 'rendah',
    'blackgram': 'rendah',
    'kidneybeans': 'rendah',
    'pigeonpeas': 'rendah',
    'mothbeans': 'rendah',
    'pomegranate': 'rendah',
    'coconut': 'rendah',
    'jute': 'rendah'
}

# Mapping dasar
df['land_area'] = df['label'].map(land_area)
df['water_need'] = df['label'].map(water_need)

# Fungsi bantu untuk parsing min–max ha
def parse_area(area_str):
    if '–' in area_str:  # range
        parts = area_str.replace(' ha','').split('–')
        return float(parts[0]), float(parts[1])
    elif '≥' in area_str:  # minimal
        val = float(area_str.replace('≥','').replace(' ha',''))
        return val, np.nan
    else:
        val = float(area_str.replace(' ha',''))
        return val, val

# Apply parsing ke kolom
df[['land_area_min','land_area_max']] = df['land_area'].apply(
    lambda x: pd.Series(parse_area(x))
)
df.drop(['land_area'], inplace=True)

# Konversi water_need jadi numerik
water_need_level = {'rendah':1, 'sedang':2, 'tinggi':3}
df['water_need_level'] = df['water_need'].map(water_need_level)

df.drop(['water_need'], inplace=True)




X = df.drop(["label"], axis=1)
y = pd.DataFrame(df.label)
labels = df.label.unique()
def to_label(i):
  return labels[i]








oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
oh_cols = pd.DataFrame(oh_encoder.fit_transform(y))

# Return the index
oh_cols.index = y.index

y = oh_cols

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

predictions = model.predict(X_val)

accuracy = accuracy_score(y_val, predictions)

print("accuracy: ", accuracy)
print("predictions: ", predictions)


predictions = np.where(predictions)[1].tolist()

predictions = list(map(to_label, predictions))
print(predictions)

